# Loading the Data

In [0]:
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import sparknlp

In [0]:
spark = sparknlp.start()
csv_path = 'data/all_data.csv'
indeed_ads_pd = pd.read_csv(csv_path)
indeed_ads = spark.createDataFrame(indeed_ads_pd)

# Imports

In [0]:
# spark and basic imports
from pyspark.sql.functions import col, udf, regexp_extract, explode, lower, length, trim, when, regexp_replace
import re

# Reduction of Records and Texts

In [0]:
# Exploring "Full Job Description" column for irrelevant records
total_count = indeed_ads.count()
error_count = indeed_ads.filter(indeed_ads["Full Job Description"] == "Error").count()
null_count = indeed_ads.filter(indeed_ads["Full Job Description"].isNull()).count()
{"Total": total_count, "Error": error_count, "Null": null_count}

{'Total': 3635, 'Error': 2730, 'Null': 246}

In [0]:
# Keep only records with relevant text in 'Full Job Description' column
filtered_indeed_ads = indeed_ads.filter((indeed_ads["Full Job Description"].isNotNull()) & (indeed_ads["Full Job Description"] != "Error")).select("Title", "Company", "Full Job Description", "Search Word")
print(filtered_indeed_ads.count())

659


In [0]:
# Removal of urls from descriptions
opened_url_pattern = r"Opened URL.*"
general_url_pattern = r"http\S+|www\S+"

filtered_indeed_ads = filtered_indeed_ads.withColumn("Description",
    when(col("Full Job Description").contains("Opened URL"),
         trim(regexp_replace(col("Full Job Description"), opened_url_pattern, "")))
    .otherwise(col("Full Job Description"))
)

filtered_indeed_ads = filtered_indeed_ads.withColumn("Description",
    regexp_replace(col("Description"), general_url_pattern, "")
)

# Dealing with Hebrew
Explore Hebrew descriptions, extract relevant English words and translate if neccesary.

In [0]:
pip install deep-translator

  Obtaining dependency information for deep-translator from https://files.pythonhosted.org/packages/38/3f/61a8ef73236dbea83a1a063a8af2f8e1e41a0df64f122233938391d0f175/deep_translator-1.11.4-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/42.3 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from deep_translator import GoogleTranslator

def translate_to_english(text):
    if not text:
        return text  # Skip empty descriptions
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except Exception as e:
        return text  # Return original text if translation fails
    
translate_udf = udf(translate_to_english, StringType())
translated_ads = filtered_indeed_ads.withColumn("Description_English", translate_udf(col("Description")))

translated_ads.limit(20).display()

Title Company Full Job Description Search Word Description Description_English Data Analytics Apple Summary Posted: Nov 7, 2024 Role Number: 200577967 We're looking for an exceptional data analyst with a strong background in distributed data processing and a demonstrated ability to turn data into actionable insights. As a data analyst for our Storage Data Analytics team at Apple, you'll play a crucial role in developing the fastest and most efficient SSDs for our products, including the iPhone, iPad, Apple Watch, and new Apple Silicon Macs. If you're passionate about continuously improving the ways we use data to make Apple's products amazing, we want to hear from you! Description On the Storage Data Analytics team, we are responsible for performing ad hoc data analysis, development and maintenance of distributed data pipelines, as well as creation and support of analysis tools. Our team focuses on Apple’s storage solutions. It is small and nimble, able to quickly explore ideas and present them to the storage teams and to leadership. Minimum Qualifications +3 years of experience Ability to lead data investigations and analysis projects with ambiguous requirements Advanced statistic and modeling knowledge Experience visualizing and presenting analyses in tools like Jupyter and Tableau Experience programming in Python in a professional setting Familiarity with distributed computation, storage, and workflow management (e.g. Splunk, Spark, Kubernetes, Kafka, Hadoop, MapReduce, AirFlow etc.) Experience with relational databases and SQL Highly organized, creative, motivated, and passionate about achieving results Curious and have excellent analytical and problem solving skills Excellent written, verbal, and data communication skills Bachelor's or Master's degree in computer science. Preferred Qualifications Familiarity with distributed computation, storage, and workflow management (e.g. Splunk, Spark, Kubernetes, Kafka, Hadoop, MapReduce, AirFlow etc.) Opened URL: https://il.indeed.com/rc/clk?jk=0433bc7e45e694a3&bb=r6UIYw2mow7Pn8IzQwq2_7gXyX4L0a9oUPOLDIseoxAN8l0MRFFfCqpU6vSJRCmVlzxZCXcY6V-lzbB66NkEzDukGjQJgUJrBPff0HR1apFcJos6F0MbonwvLMjHhNz9&xkcb=SoDD67M33wH7H8WWaB0JbzkdCdPP&fccid=8e007f7a76f9cee5&vjs=3 data Summary Posted: Nov 7, 2024 Role Number: 200577967 We're looking for an exceptional data analyst with a strong background in distributed data processing and a demonstrated ability to turn data into actionable insights. As a data analyst for our Storage Data Analytics team at Apple, you'll play a crucial role in developing the fastest and most efficient SSDs for our products, including the iPhone, iPad, Apple Watch, and new Apple Silicon Macs. If you're passionate about continuously improving the ways we use data to make Apple's products amazing, we want to hear from you! Description On the Storage Data Analytics team, we are responsible for performing ad hoc data analysis, development and maintenance of distributed data pipelines, as well as creation and support of analysis tools. Our team focuses on Apple’s storage solutions. It is small and nimble, able to quickly explore ideas and present them to the storage teams and to leadership. Minimum Qualifications +3 years of experience Ability to lead data investigations and analysis projects with ambiguous requirements Advanced statistic and modeling knowledge Experience visualizing and presenting analyses in tools like Jupyter and Tableau Experience programming in Python in a professional setting Familiarity with distributed computation, storage, and workflow management (e.g. Splunk, Spark, Kubernetes, Kafka, Hadoop, MapReduce, AirFlow etc.) Experience with relational databases and SQL Highly organized, creative, motivated, and passionate about achieving results Curious and have excellent analytical and problem solving skills Excellent written, verbal, and data communication skills Bachelor's or Master's degree in computer science. Preferred Qualifications Familiarity with distribu

# Requirements Sections
Extract the requirements section from each record.

In [0]:
# Extract only requirements texts
requirements_headers = ["minimum requirements", "minimum qualifications", "qualifications", "required skills & background", "on your resume", "about you", "who we are looking for", "skills & experience", "what you will need", "what you need", "what you bring to the table", "what we expect", "skills and/or certifications", "requirements", "essential skills", "required skills", "required background", "skills include", "position specifications", "required", "background", "experience"]

def extract_requirements(description):
    if not description:
        return ""
    description_lower = description.lower()
    for header in requirements_headers:     # Extract by order of requirements_headers
        match = re.search(rf"{header}\s*[-:]*\s*(?![,.])(.+)", description_lower)
        if match:
            start_index = match.start(1)
            end_index = match.end(1)
            # Return the corresponding text from the original description
            return description[start_index:end_index].strip()
    return ""  # If no header is found, return an empty string

extract_requirements_udf = udf(lambda desc: extract_requirements(desc), StringType())
requirements_df = translated_ads.withColumn("Requirements_Text", extract_requirements_udf(col("Description_English")))

In [0]:
# Observe records with no extracted requirements to check for relevance
empty_requirements = requirements_df.filter(col("Requirements_Text").isNull() | (col("Requirements_Text") == ""))
display(empty_requirements.select("Description_English", "Requirements_Text"))

Description_English,Requirements_Text
"We thank all individuals for their applications. As an equal opportunity employer, we encourage applications from all qualified individuals and specifically applicants from traditionally underrepresented groups – including Indigenous persons, newcomers, and persons with disabilities – who may contribute to the continued diversification of our organization.",
"We thank all individuals for their applications. As an equal opportunity employer, we encourage applications from all qualified individuals and specifically applicants from traditionally underrepresented groups – including Indigenous persons, newcomers, and persons with disabilities – who may contribute to the continued diversification of our organization.",
"We thank all individuals for their applications. As an equal opportunity employer, we encourage applications from all qualified individuals and specifically applicants from traditionally underrepresented groups – including Indigenous persons, newcomers, and persons with disabilities – who may contribute to the continued diversification of our organization.",
"We thank all individuals for their applications. As an equal opportunity employer, we encourage applications from all qualified individuals and specifically applicants from traditionally underrepresented groups – including Indigenous persons, newcomers, and persons with disabilities – who may contribute to the continued diversification of our organization.",
"We thank all individuals for their applications. As an equal opportunity employer, we encourage applications from all qualified individuals and specifically applicants from traditionally underrepresented groups – including Indigenous persons, newcomers, and persons with disabilities – who may contribute to the continued diversification of our organization.",
"We thank all individuals for their applications. As an equal opportunity employer, we encourage applications from all qualified individuals and specifically applicants from traditionally underrepresented groups – including Indigenous persons, newcomers, and persons with disabilities – who may contribute to the continued diversification of our organization.",
"We thank all individuals for their applications. As an equal opportunity employer, we encourage applications from all qualified individuals and specifically applicants from traditionally underrepresented groups – including Indigenous persons, newcomers, and persons with disabilities – who may contribute to the continued diversification of our organization.",
"We thank all individuals for their applications. As an equal opportunity employer, we encourage applications from all qualified individuals and specifically applicants from traditionally underrepresented groups – including Indigenous persons, newcomers, and persons with disabilities – who may contribute to the continued diversification of our organization.",
"We thank all individuals for their applications. As an equal opportunity employer, we encourage applications from all qualified individuals and specifically applicants from traditionally underrepresented groups – including Indigenous persons, newcomers, and persons with disabilities – who may contribute to the continued diversification of our organization.",
"Training provided where necessary. Dot Dot Loans are looking for a Data Analyst to join our team. In this role, you will analyse data, find patterns, and help the company make better decisions. You will work with different teams to ensure we use data effectively. Responsibilities Study data to find trends and insights to help the business. Use SQL to access and extract data from databases. Create and manage easy-to-understand dashboards with tools like Tableau and Power BI. Work with teams to understand their data needs and provide solutions. Write reports that explain your findings and give useful recommendations. Help with projects by providing data analysis during different stages. Use VBA t

In [0]:
# Filter out descriptions without clear requirements to extract
requirements_df = requirements_df.filter(col("Requirements_Text").isNotNull() & (col("Requirements_Text") != ""))
print(requirements_df.count())
requirements_df.select("Description_English", "Requirements_Text").limit(10).display()

647


Description_English Requirements_Text Summary Posted: Nov 7, 2024 Role Number: 200577967 We're looking for an exceptional data analyst with a strong background in distributed data processing and a demonstrated ability to turn data into actionable insights. As a data analyst for our Storage Data Analytics team at Apple, you'll play a crucial role in developing the fastest and most efficient SSDs for our products, including the iPhone, iPad, Apple Watch, and new Apple Silicon Macs. If you're passionate about continuously improving the ways we use data to make Apple's products amazing, we want to hear from you! Description On the Storage Data Analytics team, we are responsible for performing ad hoc data analysis, development and maintenance of distributed data pipelines, as well as creation and support of analysis tools. Our team focuses on Apple’s storage solutions. It is small and nimble, able to quickly explore ideas and present them to the storage teams and to leadership. Minimum Qualifications +3 years of experience Ability to lead data investigations and analysis projects with ambiguous requirements Advanced statistic and modeling knowledge Experience visualizing and presenting analyses in tools like Jupyter and Tableau Experience programming in Python in a professional setting Familiarity with distributed computation, storage, and workflow management (e.g. Splunk, Spark, Kubernetes, Kafka, Hadoop, MapReduce, AirFlow etc.) Experience with relational databases and SQL Highly organized, creative, motivated, and passionate about achieving results Curious and have excellent analytical and problem solving skills Excellent written, verbal, and data communication skills Bachelor's or Master's degree in computer science. Preferred Qualifications Familiarity with distributed computation, storage, and workflow management (e.g. Splunk, Spark, Kubernetes, Kafka, Hadoop, MapReduce, AirFlow etc.) +3 years of experience Ability to lead data investigations and analysis projects with ambiguous requirements Advanced statistic and modeling knowledge Experience visualizing and presenting analyses in tools like Jupyter and Tableau Experience programming in Python in a professional setting Familiarity with distributed computation, storage, and workflow management (e.g. Splunk, Spark, Kubernetes, Kafka, Hadoop, MapReduce, AirFlow etc.) Experience with relational databases and SQL Highly organized, creative, motivated, and passionate about achieving results Curious and have excellent analytical and problem solving skills Excellent written, verbal, and data communication skills Bachelor's or Master's degree in computer science. Preferred Qualifications Familiarity with distributed computation, storage, and workflow management (e.g. Splunk, Spark, Kubernetes, Kafka, Hadoop, MapReduce, AirFlow etc.) Israel, Hod HaSharon Description About ThetaRay: ThetaRay is a trailblazer in AI-powered Anti-Money Laundering (AML) solutions, offering cutting-edge technology to fintechs, banks, and regulatory bodies worldwide. Our mission is to enhance trust in financial transactions, ensuring compliant and innovative business growth. Our technology empowers customers to expand into new markets and introduce groundbreaking products. Thetaray is a culture-driven company. Our values are at the heart of our success. By joining us, you'll have the opportunity to embody these values and inspire others through your actions. Why Join ThetaRay? At ThetaRay, you'll be part of a dynamic global team committed to redefining the financial services sector through technological innovation. You will contribute to creating safer financial environments and have the opportunity to work with some of the brightest minds in AI, ML, and financial technology. We offer a collaborative, inclusive, and forward-thinking work environment where your ideas and contributions are valued and encouraged. Join us in our mission to revolutionize the financial world, making it safer and more trustworthy f

In [0]:
# Define a function to count tokens
def count_tokens(text):
    return len(text.split())

count_tokens_udf = udf(count_tokens, IntegerType())
requirements_token_count = requirements_df.withColumn("Token_Count", count_tokens_udf(col("Requirements_Text")))

# Filter rows with more than 512 tokens
rows_with_more_than_512_tokens = requirements_token_count.filter(col("Token_Count") > 512)
display(rows_with_more_than_512_tokens)

Title Company Full Job Description Search Word Description Description_English Requirements_Text Token_Count null events and celebrations Job Description Data Scientist (GenAI) We're hiring an exceptional Data Scientist to join our growing team of data science & AI experts! In this role, you’ll have the opportunity to design and deploy end-to-end AIML solutions that push the boundaries of financial technology, using advanced techniques in Generative AI, NLP, and predictive modeling. You’ll tackle complex business challenges, automate manual processes and help Tipalti maintain its edge in driving innovation across the fintech landscape. The ideal candidate is a problem solver with a strong technical background who thrives on transforming complex business challenges into scalable AIML solutions. Why join Tipalti Tipalti is one of the world’s fastest-growing fintech companies. We free finance professionals to lead by modernizing the entire payables operation. We are a well-funded, late-stage start-up backed by high-profile investors. Our 2021 Series F funding round raised $270 million, valuing us at over $8.3 billion. With total funding of just over $550 million, and with more than 4000 global customers, Tipalti is one of the most valuable private fintech companies in the world. At Tipalti, we pride ourselves on our collaborative culture, the quality of our product and the capabilities of our people. Tipaltians are passionate about the work they do, and keen to get the job done. Tipalti oﬀers competitive benefits, a flexible workplace, career coaching, and an environment where diverse individuals can thrive and make an impact. Our culture ensures everyone checks their egos at the door and stands ready to reach for success together. Founded in Israel in 2010, Tipalti is a global business headquartered in the San Francisco Bay Area (Foster City) with offices in Tel Aviv, Plano, Toronto, Vancouver, London, Amsterdam and Tbilisi. In This Role, You Will Be Responsible For End-to-End Solution Development Designing, developing, and deploying robust AIML solutions that address key challenges in the fintech space, from data preprocessing through production deployment. Gen AI & NLP Applications Building and fine-tuning generative AI models for innovative applications, including LLMs for text generation, summarization, categorization and document processing. Applying NLP techniques to analyze unstructured data, uncover insights, and enable capabilities like sentiment analysis, entity recognition, and text classification. Predictive Modeling Creating predictive models that forecast critical business outcomes, identify trends, and provide actionable insights to optimize decision-making. Model Deployment and Optimization Implementing best practices for model deployment, monitoring, and continuous improvement to ensure high performance and scalability of AI solutions in production. Cross-Functional Collaboration Partnering with key stakeholders across departments to understand business requirements, align on objectives, and deliver impactful solutions. About You Bachelor’s degree in Computer Science, Engineering, Mathematics, Physics, Statistics or a related field, with focus on AIML. 3+ years of hands-on experience in data science or AI engineering, ideally within the fintech industry or a related sector, with a track record of deploying impactful AIML solutions. Strong technical skills Proficiency in Python and SQL for data manipulation, analysis, and model building. Deep experience with LLMs and prompt engineering, including systematic evaluation of enterprise AI solutions Experience with building RAG systems and Conversational chatbots - Advantage Strong experience with machine learning libraries such as TensorFlow, PyTorch, or scikit-learn. Experience with cloud platforms (AWS, GCP, or Azure) for model deployment and MLOps practices. Analytical Mindset You’re adept at statistical analysis, understand key machine learning algorithms, and ca